# Week 6 Lab Exercises:  
## Classful  vs Classless Addressing

|Feature	|Classful Addressing	|CIDR (Classless)|
|-----------|----------------------|-----------------|
|Address Allocation|	Fixed (A, B, C)	|Flexible (Custom Prefixes)|
|Efficiency	|Wastes IPs (Rigid Classes)|	Efficient (Only Allocates Needed)|
|Subnet Mask	|Implicit (Based on Class)	|Explicit (e.g., /24 for 255.255.255.0)|


 ## DHCP DORA Process  
Discover: Client broadcasts to find DHCP servers.  
Offer: Server responds with an available IP.  
Request: Client accepts the offer.  
Acknowledge: Server confirms the lease.  


In [ ]:
# Exercise 1: Extended IP Analysis Tool
# Objective: Calculate subnet details for a given IP/CIDR

import ipaddress

def analyse_ip(ip_str):
    ip = ipaddress.ip_interface(ip_str)
    network = ip.network
    print(f"Address: {ip.ip}")
    print(f"Network: {network}")
    print(f"Netmask: {ip.netmask}")
    print(f"Broadcast: {network.broadcast_address}")
    print(f"First Host: {network.network_address + 1}")
    print(f"Last Host: {network.broadcast_address - 1}")
    print(f"Usable Hosts: {network.num_addresses - 2}")
    print(f"Is Private: {ip.ip.is_private}")


analyse_ip('192.168.1.1/24')

In [ ]:
# Exercise 2: Analyze Local IP Address
# Objective: Check if your IP is private/public and get network details

import socket
import ipaddress

hostname = socket.gethostname()
local_ip = socket.gethostbyname(hostname)
print(f"Your IP: {local_ip}")

# Analyse
try:
    analyse_ip(f"{local_ip}/24")  # Assumes /24 CIDR
except ipaddress.AddressValueError:
    print("Invalid IP/CIDR. Adjust the CIDR prefix (e.g., /24).")

In [ ]:
# Exercise 3: University Website IP Analysis
# Objective: Resolve "www.gold.ac.uk" and analyze its IP.

import socket
import ipaddress

def analyse_website(domain):
    try:
        ip = socket.gethostbyname(domain)
        print(f"{domain} IP: {ip}")
        analyse_ip(f"{ip}/24")  # Adjust CIDR if needed
    except socket.gaierror:
        print("DNS resolution failed. Check the domain name.")

analyse_website("www.gold.ac.uk")

In [ ]:
# Exercise 4: Subnetting Plan
# Objective: Allocate subnets for 4 departments

import ipaddress

def calculate_subnet(hosts, network):
    prefix = 32 - (hosts + 2).bit_length()  # +2 for networks/broadcasting
    subnets = list(network.subnets(new_prefix=prefix))
    return subnets[0]

base_network = ipaddress.ip_network('172.16.0.0/16')

# Calculate subnets
engineering = calculate_subnet(30, base_network)
marketing = calculate_subnet(15, base_network)
finance = calculate_subnet(10, base_network)
hr = calculate_subnet(5, base_network)

print(f"Engineering: {engineering}")
print(f"Marketing: {marketing}")
print(f"Finance: {finance}")
print(f"HR: {hr}")

In [ ]:
# Exercise 5: DHCP DORA Simulation
# Objective: Simulate DHCP server/client using TCP

# DHCP Server

import socket
import threading

server_ip = "0.0.0.0"
server_port = 6700
ip_pool = ["192.168.1.100", "192.168.1.101", "192.168.1.102"]
leases = {}

def handle_client(client_socket):
    try:
        data = client_socket.recv(1024).decode()
        if "DISCOVER" in data:
            print("[SERVER] Received DISCOVER")
            if ip_pool:
                offered_ip = ip_pool.pop(0)
                client_socket.send(f"OFFER:{offered_ip}".encode())
                print(f"[SERVER] Sent OFFER: {offered_ip}")
                request = client_socket.recv(1024).decode()
                if "REQUEST" in request:
                    leases[offered_ip] = "Leased"
                    client_socket.send(f"ACK:{offered_ip}".encode())
                    print(f"[SERVER] Sent ACK: {offered_ip}")
    except Exception as e:
        print(f"[SERVER] Error: {e}")
    finally:
        client_socket.close()

def run_dhcp_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((server_ip, server_port))
    server.listen(5)
    print(f"DHCP Server running on {server_ip}:{server_port}...")
    while True:
        client, addr = server.accept()
        threading.Thread(target=handle_client, args=(client,)).start()

run_dhcp_server()

In [ ]:
# DHCP client
import socket

def dhcp_client():
    try:
        client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client.connect(("localhost", 6700))
        client.send(b"DISCOVER")
        print("[CLIENT] Sent DISCOVER")
        
        offer = client.recv(1024).decode()
        if "OFFER" in offer:
            offered_ip = offer.split(":")[1]
            print(f"[CLIENT] Received OFFER: {offered_ip}")
            client.send(f"REQUEST:{offered_ip}".encode())
            
            ack = client.recv(1024).decode()
            print(f"[CLIENT] Received ACK: {ack}")
    except Exception as e:
        print(f"[CLIENT] Error: {e}")
    finally:
        client.close()

dhcp_client()